In [1]:
import os
import time
import numpy as np
import pandas as pd
import random
import math
from pylab import rcParams
rcParams['figure.figsize'] = 20, 15

# import TrajectoryImprovementLibrary. as TIL

import plotly.graph_objs as go
import plotly.express as px

from sklearn.preprocessing import MinMaxScaler

import folium

In [2]:
trajectory1 = np.load( os.getcwd() + '/SampleTraj/Trj_0.npy' )
trajectory2 = np.load( os.getcwd() + '/SampleTraj/Trj_1.npy' )
trajectory3 = np.load( os.getcwd() + '/SampleTraj/Trj_2.npy' )
trajectory4 = np.load( os.getcwd() + '/SampleTraj/Trj_3.npy' )

In [4]:
import pyximport
pyximport.install(reload_support=True)
import cTrajectoryImprovementLibrary as TIL

/home/petar/snap/jupyter/common/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/petar/Fakultet/Semester 8/Biologically inspired computing/Proekt/cTrajectoryImprovementLibrary.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)

Error compiling Cython file:
------------------------------------------------------------
...

def Algorithm_Random( double[][4] trajectory, bint[] variablePoints, auxParameters ):

    TRAJECTORY_LENGTH = sizeof(trajectory)/sizeof(trajectory[0])

    cdef double[][4] bestTraj = <double *> malloc(TRAJECTORY_LENGTH*4*sizeof(double)) 
                    ^
------------------------------------------------------------

cTrajectoryImprovementLibrary.pyx:54:21: Variable type 'double [][4]' is incomplete

Error compiling Cython file:
------------------------------------------------------------
...

def Algorithm_Random( double[]

ImportError: Building module cTrajectoryImprovementLibrary failed: ["distutils.errors.CompileError: command 'gcc' failed: No such file or directory: 'gcc'\n"]

In [3]:
def getNeighbours( point, n=1000, STD_DEV=.001 ):
    neighbours = []

    for _ in range( n ):

        neighbour = np.copy( point )

        neighbour[1] += np.random.normal( 0, STD_DEV )
        neighbour[2] += np.random.normal( 0, STD_DEV )

        neighbours.append(neighbour)

    return neighbours

In [4]:
def aux_CalculateTrajectoryError( trajectory, variablePoints ):
    
    totalError = 0.0
    
    for k in range( np.size( variablePoints ) ):
        
        if variablePoints[ k ]:
            single_point_error = aux_CalculateErrorSinglePoint( trajectory, k )
            
            if ~np.isnan(single_point_error):

                totalError += single_point_error
                
    
    return totalError




def aux_CalculateErrorSinglePoint( trajectory, targetPoint ):
    
    
    v_m1 = aux_haversine( trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ], trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ] )
    s_m1 = v_m1 / ( trajectory[ targetPoint, 0 ] - trajectory[ targetPoint - 1, 0 ] )
    
    
    v_m2 = aux_haversine( trajectory[ targetPoint - 2, 2 ], trajectory[ targetPoint - 2, 1 ], trajectory[ targetPoint - 1, 2 ], trajectory[ targetPoint - 1, 1 ] )
    s_m2 = v_m2 / ( trajectory[ targetPoint - 1, 0 ] - trajectory[ targetPoint - 2, 0 ] )
    
    
    v_p1 = aux_haversine( trajectory[ targetPoint, 2 ], trajectory[ targetPoint, 1 ], trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ] )
    s_p1 = v_p1 / ( trajectory[ targetPoint + 1, 0 ] - trajectory[ targetPoint, 0 ] )
    
    
    v_p2 = aux_haversine( trajectory[ targetPoint + 1, 2 ], trajectory[ targetPoint + 1, 1 ], trajectory[ targetPoint + 2, 2 ], trajectory[ targetPoint + 2, 1 ] )
    s_p2 = v_p2 / ( trajectory[ targetPoint + 2, 0 ] - trajectory[ targetPoint + 1, 0 ] )
    

    return np.std( [ v_m2, v_m1, v_p1, v_p2 ] ) / np.mean( [ v_m2, v_m1, v_p1, v_p2 ] )



def aux_haversine(lon1, lat1, lon2, lat2): # haversine is the angle distance between two points on a sphere 
    
    from math import radians, cos, sin, asin, sqrt

    R = 6372.8

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [12]:
def Genetic_Algorithm( originalTraj, variablePoints, auxParameters ):
    # 1. Inicijalna populacija: treba za variablePoints deka so se True, da izgeneriram odreden broj na edinki
    # 2. So fitnes funkcijata, u taj slucaj aux_CalculateErrorSinglePoint, za sekoja ednika da presmetam kolku e dobra
    # 3. Ko ke presmetam fitness, treba da go zacuvam toa vo dict i da gi sortiram spored vrednosta. Prvata polovina ke bide odbrana za ponatamu.
    # 4. treba pomegju edinkite nekako da napravam crossover t.e. nekako da gi vkrstam za da dobijam duplo
    # 5. Treba da napravam so odreden procent od novite edinki da se imat mutirano


    bestTraj = np.copy( originalTraj )

    best_units = dict()
    
    print(f'Best trajectory error: {aux_CalculateTrajectoryError(bestTraj, variablePoints)}')

    for iter in range(auxParameters[ 'numIters' ]):

#         print('STEP 1')

        d = dict()

        for k in range(len( originalTraj )):                            # STEP 1

            if variablePoints[k]:

                d[k] = getNeighbours(originalTraj[k])


        index_to_errors = dict()


#         print('STEP 2')

        for key in d.keys():                                            # STEP 2

            tTraj = np.copy( bestTraj )

            individual_errors = dict()

            for individual in d[key]:

                tTraj[key] = individual

                error = aux_CalculateTrajectoryError(tTraj, variablePoints)

                individual_errors[individual.tobytes()] = error

                if key in best_units.keys():

                    tTraj[key] = best_units[key]

                    best_error = aux_CalculateTrajectoryError(tTraj, variablePoints)

                    if best_error > error:

                        best_units[key] = individual

                else:

                    best_units[key] = individual

            index_to_errors[key] = individual_errors.copy()

        index_to_errors_sorted = dict()
        selected_individuals = dict()


#         print('STEP 3')

        for key in index_to_errors.keys():                              # STEP 3

            index_to_errors_sorted[key] = sorted(index_to_errors[key].items(), key=lambda x: x[1], reverse=False)

            selected_individuals[key] = dict(list(index_to_errors_sorted[key])[:int(len(index_to_errors_sorted[key])/2)])


#         print('STEP 4')

        for key in selected_individuals.keys():                         # STEP 4
            
            #treba da dojdam do edinkite, da gi vratam vo float64 ndarray format od bytes, error ne mi treba veke
            # treba da gi zemam dva po dva elementite, i da gi zamenam mestata na lon koordinatite
            
#             print(f'KEY: {key}, VALUE: {value}')

            keys = selected_individuals[key].keys()

            keys = np.array([np.frombuffer(x) for x in keys])

            for k in range(0, len(keys), 2):
                            
                keys[k][2], keys[k + 1][2] = keys[k + 1][2], keys[k][2]

#                 print('STEP 5')

                if random.random() < .2:                                # STEP 5

                    keys[k][1] += np.random.normal(0, .001)
                    keys[k][2] += np.random.normal(0, .001)
                
#             print(type(keys))
            print(keys[0])

    for key, value in best_units.items():

        bestTraj[key] = value

    return bestTraj

In [5]:
targetPoints = []

targetPoints.append(np.array(range(297, 302)))
# targetPoints.append(np.array(range(420, 470)))

targetPoints = np.hstack(targetPoints)

In [6]:
variablePoints = np.zeros( ( np.size( trajectory1, 0 ) ), dtype = bool )
variablePoints[targetPoints] = True

In [8]:
auxParameters = { 'numIters': 1 }

In [13]:
start_time = time.time()

correctedTraj1 = Genetic_Algorithm( trajectory1, variablePoints, auxParameters )

exec_time1 = time.time() - start_time

Best trajectory error: 5.1923382765446
[1.64955069e+09 3.96514917e+01 1.39597749e+02 1.21920000e+04]
[1.6495507e+09 3.9220291e+01 1.3938027e+02 1.2184380e+04]
[1.64955070e+09 3.91999686e+01 1.39370663e+02 1.21920000e+04]
[1.64955071e+09 3.91931557e+01 1.39670805e+02 1.21920000e+04]
[1.64955072e+09 3.91976032e+01 1.39367016e+02 1.21920000e+04]


In [11]:
print(exec_time1/60)

8.773339148362478


# Local Beam Algorithm

In [8]:
def Algorithm_Local_Beam( originalTraj, variablePoints, beta=3 ):
    
    bestTraj = originalTraj
    Error = aux_CalculateTrajectoryError(originalTraj, variablePoints)
    
    print(Error)

    tier_to_paths = dict()

    for k in range(len(originalTraj)):

        tmp_path = np.copy(bestTraj)

        if variablePoints[k]:
            
            print(f'Variable point {k}')

            # vo tier_to_paths dictionary da inicijaliziram na tocka k u traektorijata prazna numpy.array

            tier_to_paths[k] = []

            # za sekoj variable point da izgeneriram novi points

            neighbours = getNeighbours(originalTraj[k])

            # za sekoj izgeneriran nov point da presmetam haversine distance

            for neighbour in neighbours:

                tmp_path[k] = neighbour

                error = aux_CalculateTrajectoryError(tmp_path, variablePoints)

#                 if error < Error:
                    
#                     print('Better error, new neighbour added.')

                tier_to_paths[k].append(np.array([neighbour, error]))

            # najdobrite beta za k variable point da gi zacuvam 
            tier_to_paths[k] = sorted(tier_to_paths[k], key=lambda x: x[1], reverse=False)[:beta]
            
            print(tier_to_paths[k])
            
#             if len(tier_to_paths[k]) > 0:
            if tier_to_paths[k][0][1] < Error:

                tmp_path[k] = tier_to_paths[k][0]

                bestTraj = np.copy(tmp_path)
                Error = tier_to_paths[k][0][1]

    return bestTraj

In [9]:
Algorithm_Local_Beam(trajectory1, variablePoints)

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars
/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


nan
Variable point 260


KeyboardInterrupt: 

# Ant colony algorithm

In [91]:
'''
ZABELESKA: taj algoitam ke gi izmine site dadeni tocki, i ke vrate najkratok pat so site tocki.
'''

'\nZABELESKA: taj algoitam ke gi izmine site dadeni tocki, i ke vrate najkratok pat so site tocki.\n'

In [ ]:
#tuka treba da napravam distance matrix
# treba da presmetam distanca poedinecno za tockite, taka so ednata oska ke bide
# tockata

In [16]:
'''
Kako ke ga presmetam distance matricata?

Ke izgeneriram tocki za site variable points i ke gi zacuvam
fakticki ke presmetam distanca od tocka do tocka, od novite izgenerirani neighbours
Kako za distanca ke koristam trajectory loss

Distancata ke go pustam na par od tocki, za podatocite da mi izlezat kako u tabela, 
na ednata strana site mozni za tocka 1 na drugata site mozni za tocka 2

Ka ke dobijam matrica na distanca ke go pustam algoritmo samo na tie dve tocki,
site drugi ke se isti tocki.

timestep lat lon elevation
[]       [] []   []



'''


def distances_matrix(origianlTraj, var_point_to_neighbours, k):
    # za sekoj par tocki, treba da presmetam distance matrix
    
    # treba da gi zemam neighbours za k i k+1
    # u dvoen ciklus da presmetam haversine distance za site parovi pomegju sebe
    
    points1 = var_point_to_neighbours[k]
    points2 = var_point_to_neighbours[k+1]
    
    distance_matrix = []
    
    for point1 in points1:
        
        tmp_path = np.copy(origianlTraj)
        tmp = []
        
        for point2 in points2:
            
            tmp_path[k] = point1
            tmp_path[k+1] = point2
            
            tmp.append(aux_CalculateTrajectoryError(tmp_path, variablePoints))
            
        distance_matrix.append(np.array(tmp))
        
    return np.array(distance_matrix)

'''
izlezo na taa funkcija e ednite tocki se na ednata oska, drugite tocki se na drugata
'''

In [75]:
class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):

        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        
        print(self.distances)
        print(self.pheromone)

        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)

        for _ in range(self.n_iterations):

            all_paths = self.gen_all_paths()
            
            self.spread_pheronome(all_paths, self.n_best)
            
            shortest_path = min(all_paths, key=lambda x: x[1])

            if shortest_path[1] < all_time_shortest_path[1]:
                
                all_time_shortest_path = shortest_path     
                   
            self.pheromone = self.pheromone * self.decay   
            
        print(all_time_shortest_path)

        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best):

        sorted_paths = sorted(all_paths, key=lambda x: x[1])

        for path, dist in sorted_paths[:n_best]:

            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):

        
        total_dist = 0

        for move in path:

            total_dist += self.distances[move]

        return total_dist

    def gen_all_paths(self):

        all_paths = []

        for i in range(self.n_ants):

            path = self.gen_path(0)
            
            all_paths.append((path, self.gen_path_dist(path)))
        
        return all_paths

    def gen_path(self, start):

        path = []
        visited = set()
        visited.add(start)
        prev = start

        for i in range(len(self.distances) - 1):
            
            print(f'Prev: {prev}')

            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)

 
        return path

    def pick_move(self, pheromone, dist, visited):

        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
#         move = np.argmax(norm_row)
        move = np.random.choice(self.all_inds, 1, p=norm_row)[0]

        return move


In [68]:
def Algorithm_Ant_Colony( originalTraj, variablePoints, auxParameters, n_ants=1, n_best=1, decay=.95, alpha=1, beta=1 ):

    bestTraj = np.copy(originalTraj)
    Error = aux_CalculateTrajectoryError(originalTraj, variablePoints)
    
    var_point_to_neighbours = dict()

    for k in range(len(originalTraj)):

        if variablePoints[k]:
            
            var_point_to_neighbours[k] = getNeighbours(originalTraj[k])
    
    
    for k in range(len(originalTraj)):
        
        if variablePoints[k]:
            
            if (k + 1) in var_point_to_neighbours.keys():
                
                distance_matrix = distances_matrix(originalTraj, var_point_to_neighbours, k)
                
                ac = AntColony(distance_matrix, 1, 1, 1, .95, 1, 1)
                
                shortest_path = ac.run()
                
#                 print(shortest_path)
                print(f'Shortest path: {shortest_path}')
                
                return -1
                
                error = aux_CalculateTrajectoryError(shortest_path, variablePoints)

                if error < Error:

                    bestTraj = np.copy(shortest_path)

                    Error = error

    return bestTraj 

In [11]:
targetPoints = []

targetPoints.append(np.array(range(260, 305)))
targetPoints.append(np.array(range(420, 470)))

targetPoints = np.hstack(targetPoints)

In [12]:
variablePoints = np.zeros( ( np.size( trajectory1, 0 ) ), dtype = bool )

In [13]:
for i in range(len(variablePoints)):
    if i in targetPoints:
        variablePoints[i] = True

In [14]:
auxParameters = { 'numIters': 1 }

In [74]:
start_time = time.time()

correctedTraj1 = Algorithm_Ant_Colony( trajectory1, variablePoints, auxParameters )

exec_time1 = time.time() - start_time

[[36.20150165 36.28076001 37.31126719 35.96241087 36.37978939]
 [36.72250353 36.527786   37.43732757 36.90374621 36.45809392]
 [36.31942246 36.38717374 37.22442384 36.06354445 36.36899706]
 [36.08029719 36.12645996 37.44408922 35.93411131 36.22111489]
 [36.18419779 36.2848273  37.45226132 35.8534959  36.14943945]]
[[0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]]
Prev: 0
Prev: 1
Prev: 3
Prev: 4
Shortest path: ([(0, 1), (1, 3), (3, 4), (4, 2)], 146.85788243384533)


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


In [70]:
'''
nacino na kojso e napravena matricata na distanca u primero e site tocki se i na dvete oski
'''

distances = np.array([[np.inf, 2, 2, 5, 7],
                      [2, np.inf, 4, 8, 2],
                      [2, 4, np.inf, 1, 3],
                      [5, 8, 1, np.inf, 2],
                      [7, 2, 3, 2, np.inf]])

In [90]:
ant_colony = AntColony(distances, 1, 1, 1, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()

[[inf  2.  2.  5.  7.]
 [ 2. inf  4.  8.  2.]
 [ 2.  4. inf  1.  3.]
 [ 5.  8.  1. inf  2.]
 [ 7.  2.  3.  2. inf]]
[[0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]
 [0.2 0.2 0.2 0.2 0.2]]
Prev: 0
Prev: 1
Prev: 4
Prev: 3
([(0, 1), (1, 4), (4, 3), (3, 2)], 7.0)


In [55]:
2 + 3 + 2 + 8

15

In [30]:
def Algorithm_Ant_Colony( originalTraj, variablePoints, auxParameters, n_ants=1, n_best=1, decay=.95, alpha=1, beta=1 ):
    '''
    VARIJATNA 1
    u ciklus ke gi izminam site tocki
    za tie so se variable point ke im izgeneriram novi tocki
    ke gi najdam site megjusebni haversine distances
    po ant colony algoritamo ke gi najde patekite so najgolema verojatnost,
    na taa pateka ako ima pomalecok haversine distance od postoeckata pateka, ke ga zemam taa
    '''

    '''
    VARIJATNA 2
    u ciklus ke gi izminam site tocki
    za tie so se variable point ke im izgeneriram novi tocki
    ke gi najdam haversine distances pomegju prvata variable point i site izgenerirani tocki
    i haversine distance od izgeneriranite tocki do slednata
    po ant colony algoritamo ke gi najde patekite so najgolema verojatnost
    na taa pateka ako ima pomalecok haversine distance od postoeckata pateka, ke ga zemam taa
    '''

    bestTraj = np.copy(originalTraj)
    Error = aux_CalculateTrajectoryError(originalTraj, variablePoints)

    for k in range(len(originalTraj)):

        if variablePoints[k]:
            
            all_points = []

            all_points.append(originalTraj[k])
            all_points.append(getNeighbours(originalTraj[k]))
            all_points.append(originalTraj[k+1])


            distances = distances_matrix(all_points)

            ant_colony = AntColony(distances)

            shortest_path = ant_colony.run()

            error = aux_CalculateTrajectoryError(shortest_path, variablePoints)

            if error < Error:

                bestTraj = np.copy(shortest_path)

                Error = error

    return bestTraj 



def distances_matrix(points):
    l = []

    for i in range(len(points)-1):
        
        tmp = []
        
        for j in range(1, len(points)-1, 1):

            distance = aux_CalculateErrorSinglePoint(points[i], points[j])

            tmp.append(distance)

        l.append(np.array(tmp))

    return np.array(l)




class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):

        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):

        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)

        for _ in range(self.n_iterations):

            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])

            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path     
                   
            self.pheromone = self.pheromone * self.decay   

        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best):

        sorted_paths = sorted(all_paths, key=lambda x: x[1])

        for path, dist in sorted_paths[:n_best]:

            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):

        
        total_dist = 0

        for move in path:

            total_dist += self.distances[move]

        return total_dist

    def gen_all_paths(self):

        all_paths = []

        for i in range(self.n_ants):

            path = self.gen_path(0)
            
            all_paths.append((path, self.gen_path_dist(path)))
        
        return all_paths

    def gen_path(self, start):

        path = []
        visited = set()
        visited.add(start)
        prev = start

        for i in range(len(self.distances) - 1):

            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)

 
        return path

    def pick_move(self, pheromone, dist, visited):

        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = max(norm_row)

        return move


    
